# **Importaciones**

In [ ]:
# importa las librerías necesarias
import numpy as np                         # cálculo numérico
import pandas as pd                        # manipulación de datos
import matplotlib.pyplot as plt            # visualización
import seaborn as sns                      # visualización de datos avanzada
import warnings                            # alertas
from matplotlib import font_manager as fm  # gestión de tipografías
from scipy import stats                    # cálculos matemáticos y estadísticos
import statsmodels.api as sm               # modelos estadísticos
import calendar
import locale


# escalado y normalización
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
# transformaciones categóricas y codificación
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
# imputación de valores faltantes
# from sklearn.impute import SimpleImputer
# selección de características
# from sklearn.feature_selection import SelectKBest, chi2
# transformaciones avanzadas y combinación
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# división de datos
# from sklearn.model_selection import train_test_split

# desactiva las advertencias
warnings.filterwarnings('ignore')

# carga los datos de Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# **Análisis Exploratorio de Datos (EDA)**

In [ ]:
# cargar el dataset
df = pd.read_csv('/kaggle/input/sales-data/sales_data.csv')

# imprime las primeras 8 filas
df.head(8)

In [ ]:
# imprime las ultimas 8 filas
df.tail(8)

In [ ]:
# imprime la información del dataset
print(df.info())

In [ ]:
# imprime la estructura del dataset
print(df.columns.tolist())

In [ ]:
# imprime la estructura del dataset
print(df.shape)

In [ ]:
# agrupa por categoría y suma el total de ventas
print(df.groupby('category')['sales'].sum().round(2).sort_values(ascending = False))

In [ ]:
# agrupa por categoría y suma el total de la cantidad
print(df.groupby('category')['quantity'].sum().round(2).sort_values(ascending = False))

In [ ]:
# agrupa por categoría y suma el total de ganancias
df.groupby('category')['profit'].sum().round(2).sort_values(ascending = False)

In [ ]:
# agrupa por categoría y obtiene la media de ganancias
df.groupby('category')['profit'].mean().round(2).sort_values(ascending = False)

In [ ]:
# agrupa por categoría y obtiene la media de ganancias
df.groupby('category')['sales'].mean().round(2).sort_values(ascending = False)

In [ ]:
# agrupa por categoría y obtiene el promedio de ganancias
df.groupby('category')['profit'].mean().round(2).sort_values(ascending = False)

# **Visualización de datos**

In [ ]:
# Configurar tamaño de los gráficos
plt.rcParams['figure.figsize'] = [6, 4]

# define el estilo de los gráficos
plt.style.use('default')

# Configurar paleta de colores
sns.set_palette('viridis')

# Tamaño de letra
plt.rcParams['font.size'] = 12

# Ruta a la fuente personalizada
font_path = '/kaggle/input/font-palatino-linotype/PALAB.TTF'

# Registrar la fuente en matplotlib
font_prop = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)  

# Configurar como fuente predeterminada para todos los textos (títulos, etiquetas y números)
plt.rcParams['font.family'] = font_prop.get_name()

plt.rcParams.update({
    "font.family": font_prop.get_name(),
    "font.weight": "bold",
    "axes.titleweight": "bold"
   # "axes.labelweight": "bold",       
   # "axes.titleweight": "bold",
   # "lines.linewidth": 1.5, 
   # "lines.color": "black"
})

In [ ]:
# Convierte las fechas
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['ship_date'] = pd.to_datetime(df['ship_date'], errors='coerce')

# Extrae mes y año
df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month

# Agrupa ventas por mes
sales_by_month = df.groupby('month')['sales'].sum().reset_index()

# Para hacer más comprensible la escala
sales_by_month['sales_k'] = sales_by_month['sales'] / 1000  # ventas en miles

# Preparar nombres de meses en español
month_names = {
    1: "Ene", 2: "Feb", 3: "Mar", 4: "Abr", 5: "May", 6: "Jun",
    7: "Jul", 8: "Ago", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dic"
}
sales_by_month['mes_abbr'] = sales_by_month['month'].map(month_names)

# Orden correcto por mes
sales_by_month = sales_by_month.sort_values('month')

# Gráfico
plt.figure(figsize=(6, 4))
sns.lineplot(
    data=sales_by_month,
    x='mes_abbr',
    y='sales_k',
    marker='o',
    linewidth=1.5,
    color='black'
)

plt.title("Ventas por Mes (total acumulado)")
plt.xlabel("Mes")
plt.xticks(rotation=35)
plt.ylabel("Ventas (miles)")

plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
# Agrupamos ganancias por categoría
profit_by_category = df.groupby('category')['profit'].sum().reset_index()

plt.figure(figsize=(8,6))
sns.barplot(data=profit_by_category, x='category', y='profit', color="skyblue", edgecolor="black")

plt.title("Ganancias Totales por Categoría", fontsize=16, weight='bold')
plt.xlabel("Categoría", fontsize=12, weight='bold')
plt.ylabel("Ganancia Total", fontsize=12, weight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.show()

In [ ]:
# grafico de dispersión
plt.figure(figsize=(10,6))
sns.scatterplot(
    data=df, 
    x='discount', 
    y='profit', 
    alpha=0.5, 
    edgecolor=None
)

plt.title("Relación entre Descuento y Ganancia", fontsize=16, weight='bold')
plt.xlabel("Descuento", fontsize=12, weight='bold')
plt.ylabel("Ganancia", fontsize=12, weight='bold')
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()


In [ ]:
# Agrupa cantidad por subcategoría
qty_by_subcat = df.groupby('sub_category')['quantity'].sum().reset_index().sort_values('quantity', ascending=False)

plt.figure(figsize=(12,6))
sns.barplot(data=qty_by_subcat, x='quantity', y='sub_category', palette="viridis")

plt.title("Cantidad de Productos Vendidos por Subcategoría", fontsize=16, weight='bold')
plt.xlabel("Cantidad Total", fontsize=12, weight='bold')
plt.ylabel("Subcategoría", fontsize=12, weight='bold')
plt.grid(axis='x', linestyle='--', alpha=0.6)

plt.show()

In [ ]:
# gráfico de dispersión
plt.figure(figsize=(10,6))
sns.scatterplot(
    data=df, 
    x='shipping_cost', 
    y='profit', 
    alpha=0.5,
    color="darkblue"
)

plt.title("Relación entre Costo de Envío y Ganancia", fontsize=14, weight='bold')
plt.xlabel("Costo de Envío", fontsize=12, weight='bold')
plt.ylabel("Ganancia", fontsize=12, weight='bold')
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()

In [ ]:
# gráfico de dispersión
plt.figure(figsize=(10,6))
sns.scatterplot(
    data=df,
    x='quantity',
    y='sales',
    alpha=0.5,
    color="darkgreen"
)

plt.title("Relación entre Cantidad Vendida y Ventas", fontsize=14, weight='bold')
plt.xlabel("Cantidad de Productos", fontsize=12, weight='bold')
plt.ylabel("Ventas Totales", fontsize=12, weight='bold')
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()

In [ ]:
# Ventas por mes
df['month'] = df['order_date'].dt.month
sales_by_month = df.groupby('month')['sales'].sum().reset_index()
month_names = {1:"Ene",2:"Feb",3:"Mar",4:"Abr",5:"May",6:"Jun",
               7:"Jul",8:"Ago",9:"Sep",10:"Oct",11:"Nov",12:"Dic"}
sales_by_month['mes_abbr'] = sales_by_month['month'].map(month_names)

# Ganancia por categoría
profit_by_category = df.groupby('category')['profit'].sum().reset_index()

# Cantidad por subcategoría
qty_by_subcat = df.groupby('sub_category')['quantity'].sum().reset_index().sort_values('quantity', ascending=False).head(10)

# Crear subplots
fig, axes = plt.subplots(2, 2, figsize=(14,10))
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# Ventas por mes
sns.lineplot(data=sales_by_month, x='mes_abbr', y='sales', marker='o', color="black", ax=axes[0,0])
axes[0,0].set_title("Ventas Totales por Mes", fontsize=12, weight='bold')
axes[0,0].set_xlabel("Mes")
axes[0,0].set_ylabel("Ventas")

# Ganancias por categoría
sns.barplot(data=profit_by_category, x='category', y='profit', palette="viridis", ax=axes[0,1])
axes[0,1].set_title("Ganancias por Categoría", fontsize=12, weight='bold')
axes[0,1].set_xlabel("Categoría")
axes[0,1].set_ylabel("Ganancia")

# Cantidad por subcategoría
sns.barplot(data=qty_by_subcat, x='quantity', y='sub_category', palette="magma", ax=axes[1,0])
axes[1,0].set_title("Top 10 Subcategorías por Cantidad Vendida", fontsize=12, weight='bold')
axes[1,0].set_xlabel("Cantidad")
axes[1,0].set_ylabel("Subcategoría")

# Scatter cantidad vs ventas
sns.scatterplot(data=df, x='quantity', y='sales', alpha=0.5, color="darkgreen", ax=axes[1,1])
axes[1,1].set_title("Relación Cantidad vs Ventas", fontsize=12, weight='bold')
axes[1,1].set_xlabel("Cantidad")
axes[1,1].set_ylabel("Ventas")

plt.show()

# **Preguntas Clave**

## **¿Cuál es la venta promedio por pedido?**

In [ ]:
# calcula las ventas promedio
venta_promedio = df['sales'].mean().round(2)

print(f'Las ventas promedio por por pedido es de: {venta_promedio}')

## **¿Cuál es la ganancia total y el promedio por pedido?**

In [ ]:
# calcula la ganancia total y la ganancia promedio
ganancia_total = df['profit'].sum().round(2)
ganancia_promedio = df['profit'].mean().round(2)

# imprime los resultados
print(f'La ganancia total es de: {ganancia_total}')
print(f'La ganancia promedio es de: {ganancia_promedio}')

## **¿Qué porcentaje de pedidos tuvieron descuento?**

In [ ]:
# calcula la proporción con descuento
pct_con_descuento = round((df[df['discount'] > 0].shape[0] / df.shape[0]) * 100, 2)

# imprime el resultado
print(f'La proporción de pedidos que tuvieron descuento es: {pct_con_descuento}%')

## **¿Existe una Relación entre las Ventas y las Ganancias?**

In [ ]:
# obtiene la correlación entre ventas y ganancias
corr = df['sales'].corr(df['profit']).round(2)

# imprime el resultado
print(f'La correlación entre ventas y ganancias es: {corr}')

## **¿Cuáles fueron los 3 meses con más ventas?**

In [ ]:
#Agrupa ventas por mes
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['month'] = df['order_date'].dt.month
sales_by_month = df.groupby('month')['sales'].sum().reset_index()

# Ordena de mayor a menor
top3_months = sales_by_month.sort_values(by='sales', ascending=False).head(3)

# Imprime resultados con nombres en español
months_es = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
             "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]

for i, row in top3_months.iterrows():
    month_name = months_es[int(row['month']) - 1]
    print(f"{month_name}: {row['sales']:,.2f} en ventas")

## **¿Cuáles fueron los 3 meses con menos ventas?**

In [ ]:
# Ordena de mayor a menor
top3_months = sales_by_month.sort_values(by='sales', ascending=True).head(3)

for i, row in top3_months.iterrows():
    month_name = months_es[int(row['month']) - 1]
    print(f"{month_name}: {row['sales']:,.2f} en ventas")

# **Análisis Estadístico**

In [ ]:
# Estadísticas descriptivas generales
desc = df[['sales','profit','discount','quantity','shipping_cost']].describe()
print(desc)

In [ ]:
# Calcular estadísticas descriptivas básicas
mean_sales = df['sales'].mean()      # promedio de ventas
median_sales = df['sales'].median()  # mediana de ventas
std_sales = df['sales'].std()        # desviación estándar de ventas
max_sales = df['sales'].max()        # valor máximo de ventas
min_sales = df['sales'].min()        # valor mínimo de ventas

print(f"El promedio de ventas es: {mean_sales:.2f}")
print(f"La mediana de ventas es: {median_sales:.2f}")
print(f"La desviación estándar de ventas es: {std_sales:.2f}")
print(f"El valor máximo de ventas es: {max_sales:.2f}")
print(f"El valor mínimo de ventas es: {min_sales:.2f}")

In [ ]:
# Probabilidad: ¿Cuál es la probabilidad de que una venta sea mayor al promedio?
prob_sales_above_mean = (df['sales'] > mean_sales).mean()

print(f"La probabilidad de que una venta sea mayor al promedio es: {prob_sales_above_mean:.2%}")

In [ ]:
# Percentiles
p25 = df['sales'].quantile(0.25)  # percentil 25
p75 = df['sales'].quantile(0.75)  # percentil 75
iqr = p75 - p25                   # rango intercuartílico

print(f"El percentil 25 de las ventas es: {p25:.2f}")
print(f"El percentil 75 de las ventas es: {p75:.2f}")
print(f"El rango intercuartílico (IQR) de las ventas es: {iqr:.2f}")

In [ ]:
# Costo de envío promedio
avg_shipping_cost = df['shipping_cost'].mean()
print(f"El costo de envío promedio por orden es: {avg_shipping_cost:,.2f}")

In [ ]:
# Probabilidad de envío gratuito
p_free_shipping = (df[df['shipping_cost'] == 0].shape[0] / df.shape[0]) * 100
print(f"La probabilidad de que una orden tenga envío gratuito es: {p_free_shipping:.2f}%")

In [ ]:
# Correlación entre descuento y ganancia
corr_discount_profit = df['discount'].corr(df['profit'])
print(f"La correlación entre descuento y ganancia es: {corr_discount_profit:.3f}")

In [ ]:
# Margen de ganancia promedio
df['profit_margin'] = df['profit'] / df['sales']
avg_profit_margin = df['profit_margin'].mean()
print(f"El margen de ganancia promedio es: {avg_profit_margin:.2%}")

In [ ]:
# Probabilidad de que una orden pertenezca a la categoría más frecuente
top_category = df['category'].value_counts().idxmax()
p_top_category = (df[df['category'] == top_category].shape[0] / df.shape[0]) * 100
print(f"La categoría más frecuente es: {top_category} — "
      f"Probabilidad de que una orden pertenezca a ella: {p_top_category:.2f}%")

In [ ]:
# Intervalo de confianza para las ventas promedio
mean_sales = df['sales'].mean()
conf_int_sales = stats.t.interval(
    confidence=0.95,
    df=len(df['sales']) - 1,
    loc=mean_sales,
    scale=stats.sem(df['sales'])
)
print(f"Intervalo de confianza (95%) para las ventas promedio: {conf_int_sales[0]:,.2f} a {conf_int_sales[1]:,.2f}")

# **Pruebas Estadísticas**

In [ ]:
# Prueba de hipótesis: ¿La ganancia promedio es significativamente distinta de 0?
t_stat, p_value = stats.ttest_1samp(df['profit'], 0)
print(f"T-Statistic: {t_stat:.3f}, p-value: {p_value:.4f}")
if p_value < 0.05:
    print("Conclusión: La ganancia promedio es significativamente distinta de 0 (95% confianza).")
else:
    print("Conclusión: No hay evidencia suficiente para decir que la ganancia promedio difiere de 0.")

In [ ]:
# Regresión lineal simple: ¿Qué tan bien el descuento explica la ganancia?
X = sm.add_constant(df['discount'])  # Variable independiente
y = df['profit']                     # Variable dependiente
model = sm.OLS(y, X).fit()
print(f"Coeficiente de regresión (descuento): {model.params['discount']:.3f}")
print(f"R2 del modelo: {model.rsquared:.3f}")
print(f"p-value del coeficiente: {model.pvalues['discount']:.4f}")

In [ ]:
# Prueba Chi-cuadrado: ¿Existe relación entre la categoría de producto y el modo de envío?
contingency_table = pd.crosstab(df['category'], df['ship_mode'])
chi2, p_chi, dof, expected = stats.chi2_contingency(contingency_table)
print(f"Chi2 = {chi2:.2f}, gl = {dof}, p-value = {p_chi:.4f}")
if p_chi < 0.05:
    print("Conclusión: Existe una relación significativa entre categoría y modo de envío.")
else:
    print("Conclusión: No se detecta relación significativa entre categoría y modo de envío.")

# **Modelado de Datos**

- Proximamente...